1. 문서의 내용을 읽는다
2. 문서를 쪼갠다
   - 토큰수 초과로 답변을 생성하지 못할 수 있고
   - 문서가 길면 답변 생성이 오래 걸림
3. 임베딩 -> **벡터 데이터베이스**에 저장
4. 질문이 있을 때, 벡터 데이터베이스에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

In [39]:
# document loader
# https://python.langchain.com/v0.1/docs/integrations/document_loaders/microsoft_word/

%pip install --upgrade --quiet docx2txt langchain-community


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [40]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,  # 문맥을 겹치게 만들어서 답변의 일관성을 높임
    length_function=len
)

loader = Docx2txtLoader("./tax_raw.docx")
document_list = loader.load_and_split(text_splitter=text_splitter)

In [41]:
len(document_list)

183

In [42]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv()

embeddings = OpenAIEmbeddings(model = "text-embedding-3-large")

In [43]:
from langchain_chroma import Chroma

database = Chroma.from_documents(
    documents=document_list,
    embedding=embeddings,
    persist_directory="./chroma_db")

In [44]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'
retrieved_docs = database.similarity_search(query,  k=3)

In [45]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)

In [46]:
prompt = f"""[Identity]
- 당신은 세무사입니다.
- 당신은 세무사로서, [Context] 를 참고해서 질문에 대한 답변을 정확하게 해주세요

[Context]
{retrieved_docs}

Question: {query}
"""

In [47]:
ai_message = llm.invoke(prompt)

In [48]:
ai_message.content

'연봉 5천만원인 직장인의 소득세를 계산하기 위해서는 소득세율과 공제를 고려해야 합니다. 한국의 소득세는 누진세율로 적용되며, 2023년 기준으로 다음과 같은 세율이 적용됩니다:\n\n1. 1,200만원 이하: 6%\n2. 1,200만원 초과 4,600만원 이하: 15%\n3. 4,600만원 초과 8,800만원 이하: 24%\n\n연봉 5천만원에 대한 소득세를 계산해보면:\n\n1. 1,200만원에 대한 세금: 1,200만원 × 6% = 72만원\n2. 3,400만원(4,600만원 - 1,200만원)에 대한 세금: 3,400만원 × 15% = 510만원\n3. 400만원(5,000만원 - 4,600만원)에 대한 세금: 400만원 × 24% = 96만원\n\n이제 이 세금을 모두 합산합니다:\n\n- 총 세금 = 72만원 + 510만원 + 96만원 = 678만원\n\n여기서 기본공제(2023년 기준으로 150만원)와 근로소득공제 등을 적용해야 합니다. 근로소득공제는 연봉에 따라 다르지만, 대략적으로 5천만원의 경우 약 1,200만원 정도가 공제될 수 있습니다.\n\n따라서 최종적으로 과세표준은 다음과 같이 계산됩니다:\n\n- 과세표준 = 5,000만원 - 1,200만원(근로소득공제) - 150만원(기본공제) = 3,650만원\n\n이 과세표준에 대해 다시 세금을 계산하면:\n\n1. 1,200만원에 대한 세금: 1,200만원 × 6% = 72만원\n2. 2,450만원(3,650만원 - 1,200만원)에 대한 세금: 2,450만원 × 15% = 367.5만원\n\n총 세금 = 72만원 + 367.5만원 = 439.5만원\n\n따라서 연봉 5천만원인 직장인의 소득세는 약 439.5만원입니다. 다만, 정확한 세액은 개인의 상황에 따라 다를 수 있으므로, 세무사와 상담하여 구체적인 세액을 확인하는 것이 좋습니다.'

'연봉 5천만원인 직장인의 소득세를 계산하기 위해서는 소득세율과 공제를 고려해야 합니다. 한국의 소득세는 누진세율로 적용되며, 2023년 기준으로 다음과 같은 세율이 적용됩니다:\n\n1. 1,200만원 이하: 6%\n2. 1,200만원 초과 4,600만원 이하: 15%\n3. 4,600만원 초과 8,800만원 이하: 24%\n4. 8,800만원 초과 1억5천만원 이하: 35%\n5. 1억5천만원 초과: 38%\n\n연봉 5천만원에 대한 소득세를 계산해보면:\n\n1. 1,200만원까지: 1,200만원 × 6% = 72만원\n2. 1,200만원 초과 4,600만원까지 (3,400만원): 3,400만원 × 15% = 510만원\n3. 4,600만원 초과 5,000만원까지 (400만원): 400만원 × 24% = 96만원\n\n이제 이 세액을 모두 합산합니다:\n\n- 72만원 + 510만원 + 96만원 = 678만원\n\n따라서, 연봉 5천만원인 직장인의 소득세는 약 678만원입니다. \n\n단, 여기서 기본공제, 인적공제, 연금보험료 공제 등 다양한 공제를 적용하면 최종 세액은 달라질 수 있습니다. 공제를 고려하지 않은 단순 계산임을 유의하시기 바랍니다.'

In [52]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

/Users/hyunmui/.pyenv/versions/langchain_lecture/lib/python3.11/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [53]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=database.as_retriever(),
    chain_type_kwargs={"prompt": prompt}
)


In [ ]:
ai_message = qa({"query": query}),

In [57]:
ai_message

{'query': '연봉 5천만원인 직장인의 소득세는 얼마인가요?',
 'result': '해당 정보로는 연봉 5천만원인 직장인의 소득세를 정확히 계산할 수 없습니다. 소득세는 다양한 공제와 세율에 따라 달라지기 때문에, 구체적인 세액을 알기 위해서는 추가적인 정보가 필요합니다. 따라서 정확한 소득세 금액은 알 수 없습니다.'}